In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Importing Initial Test Data

In [2]:
books_ctlg=pd.read_csv("BOOKSCATALOGUE.csv")
purchase_history=pd.read_csv("BOOKSPURCHHISTORY.csv")
visit_history=pd.read_csv("BOOKSVISITHISTORY.csv")
users_list=pd.read_csv("USERMASTER.csv")
train_data=pd.read_csv("BOOKSMASTERTRAIN.csv")

In [3]:
users_list.head()

,UserID,SIGNUPDATE,DELETED,AGEGROUP,GENDER,STATE
0,600003,2018-09-07 01:29:02,NaN,28.0,FEMALE,NaN
1,600008,2017-11-02 20:33:20,NaN,43.0,MALE,NaN
2,600011,2017-11-21 02:03:30,NaN,35.0,MALE,NaN
3,600017,2018-02-13 12:21:22,2018-08-06 15:36:27,27.0,MALE,Massachusetts
4,600020,2017-11-13 22:29:13,NaN,53.0,MALE,Pennsylvania


# Mapping data and removing unncessary data(users_list)

In [4]:
users_list=users_list.drop(columns=['DELETED','STATE', 'SIGNUPDATE'], axis=1)
users_list.head()

,UserID,AGEGROUP,GENDER
0,600003,28.0,FEMALE
1,600008,43.0,MALE
2,600011,35.0,MALE
3,600017,27.0,MALE
4,600020,53.0,MALE


In [5]:
users_list['AGEGROUP']=np.where(users_list['AGEGROUP']>30,1,0)
users_list.head()

,UserID,AGEGROUP,GENDER
0,600003,0,FEMALE
1,600008,1,MALE
2,600011,1,MALE
3,600017,0,MALE
4,600020,1,MALE


In [6]:
users_list['GENDER']=users_list['GENDER'].map({'FEMALE':1,'MALE':0})
users_list.head()

,UserID,AGEGROUP,GENDER
0,600003,0,1
1,600008,1,0
2,600011,1,0
3,600017,0,0
4,600020,1,0


In [ ]:
Adding P

In [28]:
books_purchased=[]
books_purchased_buffer=[]
users=[]
users=users_list['UserID'].values
for i in range(5):
    books_purchased_buffer=purchase_history.loc[purchase_history['UserID'] == users[i]]
    books_purchased=books_purchased_buffer['BookID']
    
books_purchased.values
    


array([117564, 105481, 169328, 167437, 154892, 114400, 114845, 121906,
       132243, 109177, 165478, 117578], dtype=int64)